# Comparison of observations with predictions from the uncalibrated model

In [ ]:
model_path = '../data/Matsch B2/inputs'
timeout = 300
observations_path = '../data/Matsch B2/observations/obs.csv'
parameters_path = '../data/Calibration parameters/default_parameters_table.csv'
spinup = {'years': 1}
skip_months = None
default_parameters = None

In [ ]:
import pandas as pd

from dateutil.relativedelta import relativedelta

from mhpc_project.utils import date_parser
from mhpc_project.parameters import VarSoilParameters
from mhpc_project.models import VarSoilFullModel
from mhpc_project.calibration import Calibration

In [ ]:
parameters = VarSoilParameters(parameters_path, default_parameters)
model = VarSoilFullModel(model_path, timeout=timeout)
observations = pd.read_csv(observations_path,
                           parse_dates=[0],
                           date_parser=date_parser,
                           index_col=0)

if spinup:
    start_date = date_parser(model.settings['InitDateDDMMYYYYhhmm'])
    start_date = start_date + relativedelta(**spinup)
    selection = observations.index.map(lambda x: x >= start_date)
    observations = observations[selection]

if skip_months:
    selection = observations.index.map(lambda x: x.month not in skip_months)
    observations = observations[selection]

calibration = Calibration(model, parameters, observations)

In [ ]:
%%time
initial_guess = calibration.parameters.instrumentation
calibration.comparison_plots(initial_guess)